In [16]:
with open('input.txt') as f:
    lines = f.read().rstrip().split('\n')


def parse_rule(line):
    possible_rules = (line.split(': ')[1]).split(' or ')
    return (line.split(': ')[0], possible_rules)

def parse_ticket(line):
    return list(map(int, line.split(',')))


val = 'rule'

rules = {}
other_tickets = []

i = -1
while(True):
    i += 1
    if i == len(lines):
        break
        
    line = lines[i]
    
    if line == '':
        continue
    if line.startswith('your ticket'):
        val = 'myticket'
        continue
    if line.startswith('nearby tickets'):
        val = 'other_tickets'
        continue
        
    if val == 'rule':
        k, v = parse_rule(line)
        rules[k] = v
    elif val == 'myticket':
        my_ticket = parse_ticket(line)
    elif val == 'other_tickets':
        other_tickets.append(parse_ticket(line))
        
print(rules)
print()
print(my_ticket)
print()
print(other_tickets[:2])



{'departure location': ['27-180', '187-953'], 'departure station': ['47-527', '545-958'], 'departure platform': ['36-566', '572-973'], 'departure track': ['37-497', '505-971'], 'departure date': ['47-707', '719-969'], 'departure time': ['36-275', '290-949'], 'arrival location': ['31-855', '864-955'], 'arrival station': ['50-148', '158-949'], 'arrival platform': ['50-441', '467-965'], 'arrival track': ['30-648', '659-962'], 'class': ['26-470', '481-966'], 'duration': ['27-808', '818-958'], 'price': ['49-769', '784-970'], 'route': ['49-796', '809-964'], 'row': ['42-362', '383-971'], 'seat': ['34-877', '887-952'], 'train': ['31-354', '363-950'], 'type': ['39-208', '231-953'], 'wagon': ['47-736', '746-968'], 'zone': ['44-290', '310-974']}

[97, 61, 53, 101, 131, 163, 79, 103, 67, 127, 71, 109, 89, 107, 83, 73, 113, 59, 137, 139]

[[93, 566, 873, 796, 908, 899, 408, 393, 621, 571, 546, 549, 494, 631, 940, 491, 561, 108, 395, 258], [609, 949, 551, 408, 648, 723, 627, 428, 905, 665, 248, 557,

In [29]:
rules_min_max = []

def get_rule_min_max(rule):
    return list(map(int, rule.split('-')))

for field in rules.values():
    for rule_range in field:
        rules_min_max.append(get_rule_min_max(rule_range))
        
POSS_VALUES = set()

for min_val, max_val in rules_min_max:
    for i in range(min_val, max_val+1):
        POSS_VALUES.add(i)
# POSS_VALUES

In [30]:
invalid_vals = []
invalid_idxs = []

for idx, ticket in enumerate(other_tickets):
    for val in ticket:
        if val not in POSS_VALUES:
            invalid_vals.append(val)
            invalid_idxs.append(idx)
print(invalid_idxs)

[22, 24, 26, 29, 34, 38, 39, 42, 45, 51, 53, 54, 56, 65, 70, 73, 77, 79, 84, 85, 88, 94, 95, 100, 114, 127, 128, 133, 147, 153, 154, 156, 171, 178, 186, 192, 193, 199, 200, 201, 202, 211, 212, 213, 217, 227, 230, 234]


In [31]:
sum(invalid_vals)

22057

In [20]:
##Part 2:
valid_tickets = [ticket for i, ticket in enumerate(other_tickets) if i not in invalid_idxs]

In [24]:
def get_poss_vals_per_field(field):
    poss_values = set()
    rules_min_maxes = []
    for rule_range in rules[field]:
        rules_min_maxes.append(get_rule_min_max(rule_range))
    
    for min_val, max_val in rules_min_maxes:
        for i in range(min_val, max_val+1):
            poss_values.add(i)
    return poss_values
            

    
    
rules_dict = {k: get_poss_vals_per_field(k) for k in rules.keys()}
# rules_dict['departure location']

In [25]:
from collections import defaultdict

CANDIDATES = defaultdict(lambda: set())

for col in range(len(my_ticket)):
    col_vals = set(map(lambda x: x[col], valid_tickets))
    
    for rule in rules.keys():
        if col_vals.issubset(rules_dict[rule]):
            CANDIDATES[rule].add(col)


FINAL_CANDS = {}


N = len(CANDIDATES)

while(True):
    if len(FINAL_CANDS) == N:
        break
    for k, v in CANDIDATES.items():
        if len(v) == 1:
            val = list(v)[0]
            FINAL_CANDS[k] = val

            for key, vals in CANDIDATES.items():
                if val in vals:
                    CANDIDATES[key].remove(val)
            CANDIDATES.pop(k)
            break

        else: 
            continue

In [26]:
FINAL_CANDS

{'arrival track': 3,
 'class': 7,
 'price': 13,
 'type': 4,
 'row': 2,
 'wagon': 16,
 'train': 10,
 'departure platform': 0,
 'departure track': 17,
 'departure date': 19,
 'departure time': 6,
 'departure station': 9,
 'departure location': 18,
 'route': 11,
 'arrival platform': 1,
 'zone': 8,
 'seat': 5,
 'arrival location': 12,
 'duration': 14,
 'arrival station': 15}

In [27]:
departure_idxs = [v for k, v in FINAL_CANDS.items() if 'departure' in k]
departure_idxs

[0, 17, 19, 6, 9, 18]

In [28]:
myticket_vals = [my_ticket[i] for i in departure_idxs]

from functools import reduce
reduce(lambda x, y: x*y, myticket_vals)

1093427331937